in what order is OPD data stored? We know it is 1  point per corner, so that in total there are 4 OPD points for the corner wavefront sensor simulation

At first sight, it is assumed in the code that the order  in `opd.zer` should follow the `refSensorNameList`:

In [1]:
from lsst.ts.phosim.CloseLoopTask import CloseLoopTask
from lsst.ts.wep.Utility import CamType

task = CloseLoopTask()

instName = "lsst"

camType = CamType.LsstCam
cornerSensorNameList = task.getSensorNameListOfFields(instName)
cornerSensorIdList = task.getSensorIdListOfFields(instName)
refSensorNameList = []
refSensorIdList = []
for name, id in zip(cornerSensorNameList, cornerSensorIdList):
    if name.endswith("SW0"):
        refSensorNameList.append(name)
        refSensorIdList.append(id)


INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
refSensorNameList

['R00_SW0', 'R04_SW0', 'R40_SW0', 'R44_SW0']

In [3]:
refSensorIdList

[191, 195, 199, 203]

This is the ordering that is used in `ts_phosim`, and passed to eg. 

    # Set the FWHM data
    fwhm, sensor_id = self.phosimCmpt.getListOfFwhmSensorData(
            opdPssnFileName, refSensorIdList )
            
            
or 


     # Simulate to get the wavefront sensor data from WEP
    listOfWfErr = self.phosimCmpt.mapOpdDataToListOfWfErr(
                    opdZkFileName, refSensorIdList, refSensorNameList
                )
                
where opdZkFileName is `opd.zer` 

Inside eg. `mapOpdDataToListOfWfErr`,  it simply loads the `zk` with `zk = np.loadtxt(filePath)`,  and then populates the `SensorWavefrontError` object with these as 


    for sensorId, zk in zip(sensorIdList, opdZk):

        sensorWavefrontData = SensorWavefrontError(numOfZk=self.getNumOfZk())
        sensorWavefrontData.setSensorId(sensorId)
        sensorWavefrontData.setAnnularZernikePoly(zk) 
        
So the OPD order in `opd.zer`  is assumed to follow the `refSensorId` order in `refSensorIdList`.

Is that always true? 

I guess since the order of zernikes in `opd.zer` follows from `_mapOpdToZk` , and that reads the OPD raw files via `opdFileList = self._getOpdFileInDir(self.outputImgDir)`, which gives: 



In [4]:
import re
from lsst.ts.phosim.utils.Utility import getConfigDir, sortOpdFileList
def _getOpdFileInDir( opdDir):
    """Get the sorted OPD files in the directory.
    OPD: Optical path difference.
    Parameters
    ----------
    opdDir : str
        OPD file directory.
    Returns
    -------
    list
        List of sorted OPD files.
    """

    # Get the files
    opdFileList = []
    fileList = _getFileInDir(opdDir)
    for file in fileList:
        fileName = os.path.basename(file)
        m = re.match(r"\Aopd_\d+_(\d+).fits.gz", fileName)
        if m is not None:
            opdFileList.append(file)
    print(opdFileList)
    
    # Do the sorting of file name
    sortedOpdFileList = sortOpdFileList(opdFileList)
    print(sortedOpdFileList)
    
    return  sortedOpdFileList

def _getFileInDir( fileDir):
    """Get the files in the directory.
    Parameters
    ----------
    fileDir : str
        File directory.
    Returns
    -------
    list
        List of files.
    """

    fileList = []
    for name in os.listdir(fileDir):
        filePath = os.path.join(fileDir, name)
        if os.path.isfile(filePath):
            fileList.append(filePath)

    return fileList

This uses `sortOpdFileList` to  sort them https://github.com/lsst-ts/ts_phosim/blob/bf49b3865aaa4563948a262d8c6a40ccee63eb2e/python/lsst/ts/phosim/utils/Utility.py#L211 m

In [6]:
mag=17
iterN=0
outputImgDir = f'/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_{mag}/iter{iterN}/img/'
opdFileList = _getOpdFileInDir(outputImgDir)

['/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_17/iter0/img/opd_9006000_2.fits.gz', '/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_17/iter0/img/opd_9006000_1.fits.gz', '/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_17/iter0/img/opd_9006000_0.fits.gz', '/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_17/iter0/img/opd_9006000_3.fits.gz']
['/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_17/iter0/img/opd_9006000_0.fits.gz', '/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_17/iter0/img/opd_9006000_1.fits.gz', '/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_17/iter0/img/opd_9006000_2.fits.gz', '/sdf/data/rubin/u/scichris/WORK/AOS/DM-36218/wfs_grid_17/iter0/img/opd_9006000_3.fits.gz']


So the `unsorted` had `2,1,0,3` and the sorted one had `0,1,2,3`,  but how was it defined which is which? 

Lets check the `fits.gz`. Probably the sensor information is stored there:

In [8]:
from astropy.io import fits
hdul = fits.open(opdFileList[0])
hdul[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -64 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                  255 / length of data axis 1                          
NAXIS2  =                  255 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
UNITS   = 'MICRONS '                                                            

Well, it isn't! So it must be earlier that this info is stored. But I can't find this information... `opdPhoSim.log` doesn't seem to have that at all... 

I guess that follows the order  in opd field points, passed via `phosimCmpt.getOpdArgsAndFilesForPhoSim`.. 

The `idx` in `opd.inst` follows the order in `fieldX, fieldY = opdMetr.getFieldXY() `. Which is set in `CloseLoopTask. _setSkySimBasedOnOpdFieldPos` , which uses `getSensorNameListOfFields`  to query the `camera` for the `fieldX, fieldY` position of each detector in that list... 

So by that (accident?) the order would be the same... So that `0,1,2,3` stands for `00,04,40,44`  (??)






In [ ]:
This is the content of the `opd.inst` file passed to phosim. On the right the names of 
sensors that correspond to this position:

    opd  0	 1.176000	 1.176000 500.0    R44  + +    (+ + same R44
    opd  1	-1.176000	 1.176000 500.0    R40  - +    (+ - R04 
    opd  2	-1.176000	-1.176000 500.0    R00  - -    (- -  same R00 
    opd  3	 1.176000	-1.176000 500.0    R04  + -    (- + R40
    
                                                not_T    with_T   

Print the positions of sensors as produced for `closeLoopTask` https://github.com/lsst-ts/ts_phosim/blob/bf49b3865aaa4563948a262d8c6a40ccee63eb2e/python/lsst/ts/phosim/CloseLoopTask.py#L133-L146

In [12]:
import numpy as np 
from lsst.afw.cameraGeom import  FIELD_ANGLE
from lsst.ts.phosim.utils.Utility import getPhoSimPath, getAoclcOutputPath, getCamera
instName = "lsst"
fieldX, fieldY = list(), list()
camera = getCamera(instName)
for name in task.getSensorNameListOfFields(instName):
    detector = camera.get(name)
    xRad, yRad = detector.getCenter(FIELD_ANGLE)
    xDeg, yDeg = np.rad2deg(xRad), np.rad2deg(yRad)
    fieldY.append(xDeg)  # transpose for phoSim
    fieldX.append(yDeg)
    print(name, yDeg, xDeg)

R00_SW0 -1.1256388888888889 -1.1902222222222223
R00_SW1 -1.2549166666666665 -1.1903333333333332
R04_SW0 -1.1902222222222223 1.1256388888888889
R04_SW1 -1.1903333333333332 1.2549166666666665
R40_SW0 1.1902222222222223 -1.1256388888888889
R40_SW1 1.1903333333333332 -1.2549166666666665
R44_SW0 1.1256388888888889 1.1902222222222223
R44_SW1 1.2549166666666665 1.1903333333333332


But this shows me that actually first we calculate 

    R00
    R04
    R40
    R44

but given these positions, phosim is calculating

    R44
    R40
    R00
    R04 

... ??? What's going on? 

Even with transpose, the order is very different in `opd.inst` than what we would infer from the `refSensorNameList`...